In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [5]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [6]:
books["tagged_description"].to_csv(
    "tagged_description.txt",
    index=False,
    header=False,
    lineterminator="\n"  # ensures each row ends with a newline
)


In [7]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

# Split documents by newline (one line = one chunk)
text_splitter = CharacterTextSplitter(
    chunk_size=1000,   # or any number larger than your longest description
    chunk_overlap=0,
    separator="\n"
)

documents = text_splitter.split_documents(raw_documents)
print(f"{len(documents)} documents created")

Created a chunk of size 1170, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 2010, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 1186, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1059, which is longer than the specified 1000
Created a chunk of size 1272, which is longer than the specified 1000
Created a chunk of size 1637, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of s

2937 documents created


In [8]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create Chroma vector store from your documents
db_books = Chroma.from_documents(documents, embedding=embedding_model)

C:\Users\USER\AppData\Local\Temp\ipykernel_19560\857307575.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\USER\Desktop\PythonProject\books_llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install

In [23]:
query = "a book about arab"
docs = db_books.similarity_search(query, k = 1)
docs

[Document(id='7cd5ff17-c551-4712-b92a-fb346d085929', metadata={'source': 'tagged_description.txt'}, page_content='"9780231121255 The life of Us?mah ibn-Munqidh epitomizes the height of Arab civilization during the early Crusading period. These memoirs--which represent a rare first-hand account of medieval European manners, morals, politics, and medicine written by a non-European--offers new perspective and insight into an important point of military and cultural contact between the East and West. In his introduction, translator Philip Hitti writes, ""Ancient Arabic literature has preserved for us other biographies, memoirs, and reminiscences by great men, but there is hardly anything superior to this one in its simplicity of narrative, dignity, and wealth of contents and general human interest."')]

In [16]:
isbn_str = docs[0].page_content.split()[0].strip().replace('"', '')  # remove quotes
isbn_int = int(isbn_str)
book_row = books[books["isbn13"] == isbn_int]
book_row


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3811,9780802130365,0802130364,The Malady of Death,Marguerite Duras,Fiction,http://books.google.com/books/content?id=ZDHyw...,A man hires a woman to spend several weeks wit...,1988.0,3.86,60.0,1280.0,The Malady of Death,9780802130365 A man hires a woman to spend sev...


In [17]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [25]:
retrieve_semantic_recommendations("A book about programming")


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
69,9780007179817,0007179812,Microserfs,Douglas Coupland,Computer programmers,http://books.google.com/books/content?id=N3Amm...,"At computer giant Microsoft, Dan, Susan, Abe, ...",2004.0,3.88,371.0,436.0,Microserfs,"9780007179817 At computer giant Microsoft, Dan..."
266,9780060880125,0060880120,How to Be Popular,Meg Cabot,Juvenile Fiction,http://books.google.com/books/content?id=uRsWy...,Sixteen-year-old Steph Landry finds an old boo...,2006.0,3.54,288.0,19183.0,How to Be Popular,9780060880125 Sixteen-year-old Steph Landry fi...
288,9780060919887,0060919884,The Writing Life,Annie Dillard,Literary Criticism,http://books.google.com/books/content?id=leBs_...,"Annie Dillard has written eleven books, includ...",1990.0,4.03,113.0,11177.0,The Writing Life,9780060919887 Annie Dillard has written eleven...
490,9780099437598,0099437597,In Watermelon Sugar,Richard Brautigan,Social action,http://books.google.com/books/content?id=7rKgP...,iDEATH is a place where the sun shines a diffe...,2002.0,4.02,144.0,9594.0,In Watermelon Sugar,9780099437598 iDEATH is a place where the sun ...
545,9780130887016,0130887013,C++ Programmer's Notebook,James Edward Keogh;John Shapley Gray,Computers,http://books.google.com/books/content?id=tmgPA...,Covers all the C++ concepts a programmer needs...,2002.0,3.00,528.0,3.0,C++ Programmer's Notebook,9780130887016 Covers all the C++ concepts a pr...
553,9780132370455,013237045X,Data Structures and Abstractions with Java,Frank M. Carrano,Computers,http://books.google.com/books/content?id=_u28W...,"Using the latest features of Java 5, this uniq...",2007.0,3.44,998.0,34.0,Data Structures and Abstractions with Java,9780132370455 Using the latest features of Jav...
571,9780140144185,0140144188,Journal of a Novel,John Steinbeck,Literary Criticism,http://books.google.com/books/content?id=EYVy6...,Each working day from January 29 to November 1...,1969.0,4.09,182.0,998.0,Journal of a Novel: The East of Eden Letters,9780140144185 Each working day from January 29...
845,9780143036241,0143036246,The Christians and the Fall of Rome,Edward Gibbon,History,http://books.google.com/books/content?id=DAeWA...,Penguin brings you the works of the great thin...,2005.0,3.57,96.0,202.0,The Christians and the Fall of Rome,9780143036241 Penguin brings you the works of ...
846,9780143036357,0143036351,Why I Write,George Orwell,Language Arts & Disciplines,http://books.google.com/books/content?id=6I_qP...,"Throughout history, some books have changed th...",2005.0,4.03,120.0,5874.0,Why I Write,"9780143036357 Throughout history, some books h..."
859,9780143037576,0143037579,Fear and Trembling,Søren Kierkegaard,Philosophy,http://books.google.com/books/content?id=acFp6...,"The perfect books for the true book lover, Pen...",2005.0,4.00,160.0,13333.0,Fear and Trembling,9780143037576 The perfect books for the true b...
